# Using `music21` to change a full score to dict

## Step 1. Helper: convert note → token

In [ ]:
from music21 import converter, note, chord  # Import music21 classes for parsing and handling notes/chords
import json  # Import JSON for potential data serialization

# Map music21 quarterLength values to symbolic duration strings
dur_map = {
    0.25: "s",   # sixteenth note
    0.5:  "e",   # eighth note
    1.0:  "q",   # quarter note
    2.0:  "h",   # half note
    4.0:  "w"    # whole note
}

def note_to_token(n):
    """Convert a music21 note or chord to a token string with duration"""
    
    # Map the note/chord duration to its symbolic representation
    dur = dur_map.get(round(n.duration.quarterLength, 2), 
                        str(n.duration.quarterLength))  # fallback: use numeric duration
    
    if isinstance(n, note.Note):
        # Single note: "C4_q"
        return f"{n.nameWithOctave}_{dur}"
    
    elif isinstance(n, chord.Chord):
        # Chord: join all pitch names with dots, e.g., "C4.E4.G4_q"
        pitches = ".".join(p.nameWithOctave for p in n.pitches)
        return f"{pitches}_{dur}"
    
    return None  # Return None if input is neither Note nor Chord


## Step 2. Midi to text

In [ ]:
from GM_PROGRAMS import GM_PROGRAMS  # Import mapping of MIDI program numbers to instrument names

def get_instrument_name(part, idx):
    """Determine the name of an instrument for a given music21 part"""
    
    instr = part.getInstrument(returnDefault=True)  # Get the instrument object, use default if none set

    # Try using the part's explicit name
    if instr.partName:
        return instr.partName.strip()
    
    # Try using the instrument's general name
    if instr.instrumentName:
        return instr.instrumentName.strip()
    
    # Try using MIDI program number to get standard GM instrument name
    if hasattr(instr, "midiProgram") and instr.midiProgram is not None:
        return GM_PROGRAMS.get(instr.midiProgram, f"Program_{instr.midiProgram}")
    
    # Fallback: use generic track name based on index
    return f"Track_{idx+1}"


In [ ]:
from mido import MidiFile  # Import mido for reading MIDI files

def get_bpm_from_midi(midi_path):
    """
    Reads the first 'set_tempo' message from a MIDI file and returns BPM.
    Defaults to 120 BPM if no tempo message is found.
    """
    mid = MidiFile(midi_path)  # Load the MIDI file
    for track in mid.tracks:  # Iterate over all tracks
        for msg in track:  # Iterate over messages in the track
            if msg.type == 'set_tempo':  # Look for tempo messages
                # mido stores tempo as microseconds per beat
                tempo_us = msg.tempo
                bpm = 60_000_000 / tempo_us  # Convert to beats per minute
                return bpm
    return 120  # Return default BPM if no tempo message found


In [ ]:
def midi_to_dict(midi_path):
    """
    Convert a MIDI file into a dictionary mapping each instrument to a sequence of note/chord/rest tokens.
    """
    
    score = converter.parse(midi_path)  # Parse MIDI file into a music21 score
    data = {}  # Dictionary to store instrument-token mapping

    for i, part in enumerate(score.parts):  # Iterate over all parts (instruments)
        
        instr_name = get_instrument_name(part, i)  # Determine instrument name
        tokens = []  # List to store tokenized notes/chords/rests
        
        for n in part.recurse().notesAndRests:  # Iterate over all notes and rests in the part
            
            if isinstance(n, note.Rest):
                # Convert rest to token with duration
                dur = dur_map.get(round(n.duration.quarterLength, 2), str(n.duration.quarterLength))
                tokens.append(f"Rest_{dur}")
            else:
                # Convert note or chord to token
                tokens.append(note_to_token(n))
        
        data[instr_name] = tokens  # Store token sequence under instrument name
        
    return data  # Return the dictionary


## Step 3. Save txt file

In [5]:
def save_dict_to_txt(data, out_path="output.txt"):
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(json.dumps(data, indent=2))

## TEST

In [ ]:
midi_file = "../data/k520.mid"   # <<< replace with your MIDI file

output_file = "test_txt/output_2.txt"

result = midi_to_dict(midi_file)
save_dict_to_txt(result, output_file)

print(f"Saved token dict to {output_file}")

Saved token dict to output_2.txt


In [ ]:
from decoder import decode_to_midi

bpm = get_bpm_from_midi(midi_file)
decode_to_midi(output_file, "test_audio/my_music_2.mid", bpm=bpm)

Successfully decoded and saved MIDI as test_outputs/my_music_2.mid with bpm = 60.0


'test_outputs/my_music_2.mid'